In [10]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Define paths

train_csv_path = r"C:\Users\rikdu\Downloads\SIIMS Melanoma\subfolders\subfolder_1\train.csv"
validation_csv_path = r"C:\Users\rikdu\Downloads\SIIMS Melanoma\subfolders\subfolder_1\validation.csv"
augmented_images_path = r"C:\Users\rikdu\Downloads\SIIMS Melanoma\augmented images\augmented_images_v2"
original_train_images_path = r"C:\Users\rikdu\Downloads\SIIMS Melanoma\train"

# Load data
train_df = pd.read_csv(train_csv_path)
validation_df = pd.read_csv(validation_csv_path)

# Load images
def load_images(df, img_paths):
    images = []
    for img_path in img_paths:
        img = load_img(img_path, target_size=(224, 224))
        img_array = img_to_array(img)
        images.append(img_array)
    return np.array(images)

train_images = load_images(train_df, train_df['image_path'])
validation_images = load_images(validation_df, validation_df['image_path'])

# Normalize images
train_images = train_images / 255.0
validation_images = validation_images / 255.0

# Extract metadata and labels
train_metadata = train_df[['age_approx']].values  # Example: Extracting age
validation_metadata = validation_df[['age_approx']].values

# Labels
train_labels = train_df['target'].values
validation_labels = validation_df['target'].values

# Define model
inp_image = Input((224, 224, 3))
inp_metadata = Input((train_metadata.shape[1],))

def inception_block(x, base_channels=16):
    conv_1 = Conv2D(base_channels * 2, 1, 1, activation='relu')(x)
    conv_2_a = Conv2D(base_channels * 2, 1, 1, activation='relu')(x)
    conv_2_b = Conv2D(base_channels * 2, 3, 1, padding='same', activation='relu')(conv_2_a)
    conv_3_a = Conv2D(base_channels, 1, 1, activation='relu')(x)
    conv_3_b = Conv2D(base_channels, 5, 1, padding='same', activation='relu')(conv_3_a)
    mpool_4_a = MaxPooling2D(3, 1, padding='same')(x)
    conv_4_b = Conv2D(base_channels, 1, 1, activation='relu')(mpool_4_a)
    return Concatenate(axis=-1)([conv_1, conv_2_b, conv_3_b, conv_4_b])

block_1 = inception_block(inp_image)
block_2 = inception_block(block_1, base_channels=8)
block_3 = inception_block(block_2, base_channels=8)

gap = GlobalAveragePooling2D()(block_3)
flatn = Flatten()(gap)

# Metadata branch
metadata_dense = Dense(32, activation='relu')(inp_metadata)
metadata_output = Dense(16, activation='relu')(metadata_dense)

# Concatenate image and metadata branches
concat = Concatenate(axis=-1)([flatn, metadata_output])
output = Dense(1, activation="sigmoid")(concat)

model = Model(inputs=[inp_image, inp_metadata], outputs=output)

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("C:\\Users\\rikdu\\Downloads\\SIIMS Melanoma\\subfolders\\subfolder_1\\best_model.h5", 
                                   monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

# Train the model
history = model.fit(
    [train_images, train_metadata], train_labels,
    validation_data=([validation_images, validation_metadata], validation_labels),
    epochs=10,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint, reduce_lr]
)

# Save the model
model.save("C:\\Users\\rikdu\\Downloads\\SIIMS Melanoma\\subfolders\\subfolder_1\\final_model.h5")


Epoch 1/10
350/350 [==============================] - 1116s 3s/step - loss: 0.2835 - accuracy: 0.8857 - val_loss: 0.1155 - val_accuracy: 0.9643 - lr: 0.0010
Epoch 2/10
350/350 [==============================] - 1084s 3s/step - loss: 0.1503 - accuracy: 0.9601 - val_loss: 0.1076 - val_accuracy: 0.9704 - lr: 0.0010
Epoch 3/10
350/350 [==============================] - 1153s 3s/step - loss: 0.1482 - accuracy: 0.9599 - val_loss: 0.1156 - val_accuracy: 0.9736 - lr: 0.0010
Epoch 4/10
350/350 [==============================] - 1098s 3s/step - loss: 0.1407 - accuracy: 0.9621 - val_loss: 0.1024 - val_accuracy: 0.9682 - lr: 0.0010
Epoch 5/10
350/350 [==============================] - 1092s 3s/step - loss: 0.1406 - accuracy: 0.9634 - val_loss: 0.1091 - val_accuracy: 0.9643 - lr: 0.0010
Epoch 6/10
350/350 [==============================] - 1059s 3s/step - loss: 0.1371 - accuracy: 0.9640 - val_loss: 0.1024 - val_accuracy: 0.9693 - lr: 0.0010
Epoch 7/10
350/350 [==============================] - 1069